User based Recommendation (Collaborative filtering)
if user A and user B share similar profile then meal liked by User B but not rated by A are recommended to A
User-based: measure the similarity between target users and other users
Collaborative filtering systems make recommendations based on historic users’ preference for items (clicked, watched, purchased, liked, rated, etc.).

In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

df_profile = pd.read_csv('user_Profiles.csv')
df_activity = pd.read_csv('user_activity.csv')
df_profile.head(3)


,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_1,veg,manganese,heart_disease pregnancy,ketogenic_diet
1,User_3,non-veg,iron,pregnancy diabeties goitre hypertension obesi...,ketogenic_diet high_fiber_diet high_protien_d...
2,User_6,non-veg,iron,pregnancy diabeties goitre hypertension obesi...,ketogenic_diet high_fiber_diet high_protien_d...


In [4]:
df_activity.head(3)

,User_Id,Name,catagory,Veg_Non,Review,Nutrient,Disease,Diet
0,User_User_73,bok choy salad r,salad,veg,9,fiber,goitre diabeties kidney_disease,ketogenic_diet high_fiber_diet high_protien_d...
1,User_User_98,millet curd rice r,rice,veg,2,carbohydrates,diabeties goitre heart_disease kidney_disease...,high_protien_diet ketogenic_diet vegan_diet
2,User_User_55,methi turmeric tea r,tea,veg,2,manganese,heart_disease,ketogenic_diet


In [6]:
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('user_Profiles.csv')
        self.activity = pd.read_csv('user_activity.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results =pd.concat([df_results,self.df.loc[i]], ignore_index=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input])
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] #taking acitivies
        results = results[results['User_Id']!=user_id] # Excluding Activities of the Current User
        results = results.filter(['Name','Nutrient','Veg_Non','Review','Diet','Disease'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results
    

For Example user A =='User_71' Profile says :

In [7]:
profile_A = ['calcium','iron','diabeties','vegan_diet']
user_id = 'User_71' # asumming for example

finding similar users

In [8]:

ob = Recommender()
data = ob.get_features()
total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
for i in profile_A:
    d[i] = 1

final_input = list(d.values())
similar_users = ob.k_neighbor([final_input]) # pass 2d array []
similar_users

/Users/yuthikaanvitha/Desktop/SDP-2/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_34,veg,carbohydrates,diabeties,high_protien_diet vegan_diet
1,User_93,veg,carbohydrates,diabeties,high_protien_diet vegan_diet
2,User_74,veg,phosphorus,obesity,vegan_diet
3,User_96,veg,phosphorus,obesity,vegan_diet
4,User_81,non-veg,iron,pregnancy,ketogenic_diet high_fiber_diet alkaline_diet ...
5,User_55,veg,manganese,heart_disease,ketogenic_diet


recommending meals that are new to user A and reviwed by above users

In [13]:

recommendations = ob.user_based(profile_A,user_id) 
recommendations

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Name,Nutrient,Veg_Non,Review,Diet,Disease
0,paneer schezwan bread roll r,phosphorus,veg,1,vegan_diet high_fiber_diet,scurvy heart_disease
1,air fryer aloo corn cutlet r,carbohydrates,veg,6,high_protien_diet vegan_diet,diabeties
2,cheesy paneer cigar roll r,phosphorus,veg,8,vegan_diet,obesity
3,nilgiri turkey korma r,iron,veg,1,ketogenic_diet high_protien_diet vegan_diet a...,goitre pregnancy
4,tray roast leg of lamb r,iron,non-veg,10,ketogenic_diet vegan_diet alkaline_diet high_...,pregnancy
